In [5]:
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
Image.MAX_IMAGE_PIXELS = None  # Disable DecompressionBombError
import pandas as pd
import os
import matplotlib.pyplot as plt
from skimage import io, transform
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
# Size of feature maps in discriminator
ndf = 64

# Number of channels in the training images. For color images this is 3
nc = 3

In [6]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        
        self.conv1 =nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1)
        self.bn1=nn.BatchNorm2d(32)
        self.leakyreLU1=nn.LeakyReLU(0.2, inplace=True)
            
            
        self.conv2 =nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.bn2=nn.BatchNorm2d(64)
        self.leakyreLU2=nn.LeakyReLU(0.2, inplace=True)
        self.pool_1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
            
        self.conv3 =nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
            
        self.reg_1 = nn.Linear(128 * 16 * 16, 700)
        self.reg_1_1 = nn.Linear(700, 70)
            
            
        self.deconv1=nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2)
        self.bn3=nn.BatchNorm2d(64)
        self.conv4 =nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=1)
        self.bn4=nn.BatchNorm2d(64)
        self.reg_2 =nn.Linear(128 * 16 * 16, 700)
        self.reg_2_1 =nn.Linear(700, 70)
            
        
        
    def forward(self,x_1,y_1,x_2,y_2,image1,image2):
        x = self.leakyreLU1(self.bn1(self.conv1(image1)))
        x = self.leakyreLU2(self.bn2(self.conv2(x)))
        x = self.pool_1(x)
        x = self.conv3(x)
        x_1 = x.view(-1, 128 * 16 * 16)
        x_1 = self.reg_1(x_1)
        x_1 = self.reg_1_1(x_1)
        
        x = F.relu(self.bn3(self.deconv1(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x_2 = x.view(-1, 64 * 17 * 17)
        x_2 = self.reg_2(x_2)
        x_2 = self.reg_2_1(x_2)
        
        # add evaluation function
        
        return 